In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import col, regexp_extract,to_date, year, month, dayofmonth

warehouse_path = "hdfs://namenode:9000/iceberg/"
iceberg_spark_jar  = '/opt/bitnami/spark/jars/iceberg-spark-runtime-3.5_2.12-1.7.1.jar,/opt/bitnami/spark/jars/mysql-connector-j-8.0.31.jar'
catalog_name = "demo"

# Setup iceberg config
conf = SparkConf().setAppName("MyApp") \
    .set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .set(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
    .set('spark.jars.packages', "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1") \
    .set(f"spark.sql.catalog.{catalog_name}.warehouse", warehouse_path) \
    .set(f"spark.sql.catalog.{catalog_name}.type", "hadoop")\
    .set("spark.sql.defaultCatalog", catalog_name) 

# Create spark session
spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
print("done")

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/trannam/.ivy2/cache
The jars for the packages stored in: /home/trannam/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b2fb4045-e7e2-4107-b269-8420d47cbe74;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.1 in central
downloading https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.5_2.12/1.7.1/iceberg-spark-runtime-3.5_2.12-1.7.1.jar ...
	[SUCCESSFUL ] org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.1!iceberg-spark-runtime-3.5_2.12.jar (208201ms)
:: resolution report :: resolve 1899ms :: artifacts dl 208208ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

done


In [6]:
# read parquet file
df = spark.read.parquet("hdfs://namenode:9000/iceberg/db/table/data/year=2023/month=8/day=25")

# show schema
df.printSchema()

root
 |-- index: integer (nullable = true)
 |-- Company: string (nullable = true)
 |-- Valuation: string (nullable = true)
 |-- Date_Joined: date (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Selector_Investors: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



In [7]:
df.show(truncate=False)

+-----+-------+---------+-----------+-------+------+-----------------+---------------------------------------------------------+----+-----+---+
|index|Company|Valuation|Date_Joined|Country|City  |Industry         |Selector_Investors                                       |year|month|day|
+-----+-------+---------+-----------+-------+------+-----------------+---------------------------------------------------------+----+-----+---+
|153  |Zepto  |5        |2023-08-25 |India  |Mumbai|Consumer & Retail|Nexus Venture Partners, Contrary, Global Founders Capital|2023|8    |25 |
+-----+-------+---------+-----------+-------+------+-----------------+---------------------------------------------------------+----+-----+---+



In [22]:
spark.sql("""INSERT INTO db.table 
        VALUES (1000,'ByteDance', 225, to_date(2003-04-04), 'China', 'Beijing', 'Media & Entertainment', 
        'Sequoia Capital China, SIG Asia Investments, Sina Weibo, SoftBank Group',2003,4,4)""").show()
print("done")

++
||
++
++

done


In [24]:
spark.sql("""SELECT Count(*) FROM db.table""").show()
print("done")

+--------+
|count(1)|
+--------+
|     302|
+--------+

done
